In [4]:
import numpy as np

def segundosTotais(timeStamp, referenciaZero = True):
    # timeStamp deve ser uma lista com strings no formato ano-mes-dia HH:MM:SS
    
    #Separando a data e o horario do timestamp em vetores diferentes
    data_hora = np.array([string.split(" ") for string in timeStamp])
    
    data = data_hora[:,0]
    horario = data_hora[:,1]
    
    #Transformando o horario em segundos
    hora = np.empty(0)
    min = np.empty(0)
    seg = np.empty(0)
    
    horarioSeparado = np.array([stringHorario.split(":") for stringHorario in horario])#Separa o horario no formato hh:mm:ss, onde cada posição do vetor armazenará hh, mm, ss  
    hora = horarioSeparado[:, 0].astype(float)
    min = horarioSeparado[:, 1].astype(float)
    seg = horarioSeparado[:, 2].astype(float)

    hora_em_seg = hora * 3600
    min_em_seg = min * 60

    
    segTotal = hora_em_seg + min_em_seg + seg
    
    #Colacando os valores na referência de tempo zero. O primeiro valor ocorrerá em t = 0 s
    
    if referenciaZero:
        segTotal = segTotal - segTotal[0]

    return segTotal
    
    

dadoServidorModbus = np.loadtxt(fname="Modbus\\ValoresDoModbusServer_4.csv", delimiter=",", skiprows=1, dtype=str )
#print(dadoServidorModbus) ta ok
 
valorServidor = dadoServidorModbus[:,0]
timeStampServidor = dadoServidorModbus[:,1]
#timeStampServidor = np.delete(timeStampServidor, 1)


#print(valorServidor)
#print(timeStampServidor)


#segTotalServidor = segundosTotais(timeStampServidor, referenciaZero=False)
#print(segTotalServidor)

dadoDb = np.loadtxt(fname="testeGatewayDB.Teste1_4.csv", delimiter=",", skiprows=1, dtype=str )

valorDb = dadoDb[:,1]
timeStampDb = dadoDb[:,2]

indice = np.where(valorDb == valorDb[-1]) # pegando o indicde do ultimo valor pq ele repretiu no banco
indice = indice[0][0]
#print(indice)

valorDb = valorDb[:indice+1]         # No banco de dados o valor de 1521 foi perdido
timeStampDb = timeStampDb[:indice+1]

#print(valorDb)     ok
#print(timeStampDb) ok

#Pegando apenas os indices diferentes de 999 que indica o valor quando não começaram a alterar os valores no servidor
indicesValidos = np.where(valorDb != '999')


valorDb = valorDb[indicesValidos]
timeStampDb = timeStampDb[indicesValidos]
#print(valorDb)     #ta ok
#print(timeStampDb) #ta ok

#print(valorServidor)
#print(valorDb)


indicesValidos = np.empty(0)
for valor in valorDb:
    indiceValido = np.where(valorServidor == valor)[0]
    
    indicesValidos = np.append(indicesValidos, indiceValido)

indicesValidos = indicesValidos.astype(int)
#print(indicesValidos)

if(indicesValidos.size == valorServidor.size):
    print("Não foram perdidos dados no servidor")
else:
    print(f"Foram perdidos {valorServidor.size - indicesValidos.size} dados")

'''
ultimoIndiciValido = indicesValidos[-1]
indicesPerdidos = np.empty(0)
ValorIndice = 0

for i in indicesValidos:
    if ValorIndice == i:
        ValorIndice += 1
    else:
        indicesPerdidos = np.append(indicesPerdidos, ValorIndice)
'''     

    
dadosSevidorValidos = dadoServidorModbus[indicesValidos]
timeStampServidorValidos = timeStampServidor[indicesValidos]

segTotalServidor = segundosTotais(timeStampServidorValidos, referenciaZero=False)

#print(dadosSevidorValidos)
#print(timeStampServidorValidos)   

segTotalDb = segundosTotais(timeStampDb, referenciaZero=False)
#print(segTotalDb)

#Colocando ambos na referência do tempo do servidor como começando em zero

segTotalDb = segTotalDb - segTotalServidor[0]
segTotalServidor = segTotalServidor - segTotalServidor[0]

print(f"segTotalServidor = \n{segTotalServidor}")
print(f"segTotalDb = \n{segTotalDb}")

diferenca = segTotalDb - segTotalServidor
print(f"diferencaEntreServidorEDb = \n{diferenca}")


Não foram perdidos dados no servidor
segTotalServidor = 
[ 0.        0.505458  1.006211  1.50679   2.007595  2.507828  3.008116
  3.508735  4.009498  4.509843  5.010174  5.510529  6.011242  6.511819
  7.012309  7.51327   8.013718  8.514657  9.01638   9.516798 10.017412
 10.517958 11.018589 11.519722 12.020365 12.520675 13.021325 13.522122
 14.022655 14.523445 15.023996 15.524626 16.025097 16.525949 17.026245
 17.526805 18.027377 18.528189 19.028937 19.529539 20.030115 20.530675
 21.031165 21.532026 22.032475 22.533084 23.033708 23.534542 24.035272
 24.535919 25.036329 25.537226 26.037773 26.538695 27.039286 27.539918
 28.040502 28.541302 29.04189  29.542592 30.04317  30.543948 31.044444
 31.545253 32.046211 32.546835 33.047261 33.548078 34.048706 34.549298
 35.049855 35.550271 36.050788]
segTotalDb = 
[ 0.252409  0.753247  1.254054  1.755028  2.255801  2.756563  3.257019
  3.757812  4.258772  4.759635  5.260827  5.761608  6.262342  6.762754
  7.263384  7.763831  8.264474  8.765344  9.2

In [23]:
import numpy as np

dadoServidorModbus = np.loadtxt(fname="EsteiraDb.VelocidadeMotor.csv", delimiter=",", skiprows=1, dtype=str )
#print(dadoServidorModbus) ta ok
 
valor = dadoServidorModbus[:,1]
timeStampInicio = dadoServidorModbus[:,2]
timeStampRecebidoNaAplicacao = dadoServidorModbus[:,3]

#print(timeStamp)
segs = segundosTotais(timeStampInicio, False)
#print(segs)
segs2 = np.delete(segs, 0)
segs2 = np.append(segs2, 0)
#print(segs)
#print(segs2)

dir = segs2 - segs
dir = dir[:-1]
print("Amostragem tempo de inicio")
print(dir)
print(np.mean(dir))
print(np.std(dir))

segs = segundosTotais(timeStampRecebidoNaAplicacao, False)
segs2 = np.delete(segs, 0)
segs2 = np.append(segs2, 0)
dir = segs2 - segs
dir = dir[:-1]
print("Amostragem completa")
print(dir)
print(np.mean(dir))
print(np.std(dir))

segs_inicio = segundosTotais(timeStampInicio, False)
segs_aplicacao = segundosTotais(timeStampRecebidoNaAplicacao, False)

#print(segs_inicio)
#print(segs_aplicacao)

dir = segs_aplicacao - segs_inicio
print(dir)

Amostragem tempo de inicio
[0.502093 0.507642 0.490602 0.509736 0.500723 0.499087 0.510827 0.499111
 0.50104  0.498871 0.501232 0.498549 0.511029 0.499037 0.501098 0.498799
 0.501006 0.509254 0.500853 0.499143 0.501053 0.498942 0.510979 0.498926
 0.501113 0.498833 0.500924 0.509004 0.500779 0.49951  0.500815 0.499099
 0.510595 0.499146 0.500962 0.509313 0.490683 0.509161 0.500957 0.499365
 0.510978 0.498652 0.501282 0.498958 0.501077 0.498674 0.500854 0.509249
 0.501071 0.502903 0.497202 0.508716 0.490632 0.509498 0.50075  0.509312
 0.490874 0.508893 0.500875 0.49908  0.500865 0.509236 0.491469 0.508606
 0.503416 0.495919 0.514179 0.496354 0.500469 0.499773 0.500891 0.508725
 0.501142 0.49897  0.500742 0.499139 0.501187 0.508804 0.502197 0.49807
 0.501008 0.498633 0.511666 0.498455 0.501637 0.508393 0.501131 0.498822
 0.501165 0.499151 0.500302 0.499295 0.511238 0.498866 0.501673 0.498294
 0.501157 0.498998 0.510973 0.498923 0.50108  0.509043 0.501108 0.498598
 0.50155  0.498598 0.5100

In [4]:
import numpy as np 
import os


vel = np.loadtxt(fname="Ensaio_14_07\\EsteiraDb.VelocidadeEsteira100ms.csv", delimiter=",", skiprows=1, dtype=str )
freq = np.loadtxt(fname="Ensaio_14_07\\EsteiraDb.FrequenciaInversor100ms.csv", delimiter=",", skiprows=1, dtype=str )

valor = vel[:,1]
timeStampInicio = vel[:,2]
timeStampRecebidoNaAplicacao = vel[:,3]

segs = segundosTotais(timeStampRecebidoNaAplicacao, True)

FileNotFoundError: Ensaio_14_07\EsteiraDb.VelocidadeEsteira100ms.csv not found.